In [14]:
from qsprpred.data.storage.tabular.simple import PandasChemStore
from qsprpred.data.storage.tabular.hierarchical import PandasRepresentationStore
from spock.storage.tabular import SpockStorage
from graphein.molecule import plotly_molecular_graph
from qsprpred.plotting.grid_visualizers import table_to_grid
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from Bio import PDB
import os
import nglview

In [15]:
# load the correct storages from the data folder
store = SpockStorage.fromFile('../Data/temp/SpockStorage/meta.json') #(add an extra . in line 68 of the json to resolve pathing issues)
library = PandasChemStore.fromFile('../Data/temp/ChemStorage/meta.json')
representation_store = PandasRepresentationStore.fromFile('../Data/temp/ProtomerStorage/ProtomerStorage/ProtomerStorage_representations/meta.json')

In [ ]:
# Initialize mol without printing anything
for mol in library:
    mol.as_rd_mol()
    break

In [ ]:
# initialize mol_representation without printing anything
for mol in representation_store:
    if mol.representations and len(mol.representations) > 1:
        break

In [ ]:
# get summary to validate the data
store.getSummary()

In [ ]:
# visualise 2D representation of the molecules
table_to_grid(library)

In [ ]:
# get the DF of the store
df = store.getDF().reset_index(drop=True)

In [ ]:
# get the df from the promoterstorage
df2 = representation_store.getDF().reset_index(drop=True)

In [ ]:
# merge df with df2 on shared column
merged_df = pd.merge(df, df2, left_on='parent_id', right_on='ProtomerStorage_representations_ID', how='inner')
merged_df = merged_df[['parent_id_y', 'ProtomerStorage_representations_ID', 'sdf_x', 'vina_energy_total', 'SMILES_y']]
merged_df = merged_df.rename(columns={'parent_id_y': 'inchikey', 'sdf_x': 'sdf', 'SMILES_y': 'SMILES'})
merged_df['vina_energy_total'] = pd.to_numeric(merged_df['vina_energy_total'], errors='coerce')
merged_df

In [ ]:
# Group by 'inchikey', find the 2 rows with the lowest 'vina_energy_total' for each group
grouped_df = merged_df.loc[merged_df.groupby('inchikey')['vina_energy_total'].nsmallest(1).index.get_level_values(1)]
grouped_df = grouped_df.sort_values(by='vina_energy_total', ascending=True)
grouped_df

In [ ]:
# remove the rows where vina_energy_total > -7
filtered_df = grouped_df[grouped_df['vina_energy_total'] <= -4.5]
filtered_df

In [ ]:
# create a new identifyier column using inhcikey and vina_energy_total
filtered_df['identifier'] = filtered_df['inchikey'] + '_' + filtered_df['vina_energy_total'].astype(str)
filtered_df
# set it as the index
filtered_df.set_index('identifier', inplace=True)
filtered_df

In [ ]:
# get list of the first 10 SMILES
smiles_list = filtered_df['SMILES'].tolist()
smiles_list 

In [ ]:
# extract the binding pose of compound of choice. input from Parent_ID column
mol_id = 'ProtomerStorage_representations_library_63d6cf460e5042c2966f93b820609354_1'
poses = store.get_poses(mol_id=mol_id, target=store.targets[0].id)
len(poses)

In [ ]:
# visualise the binding pose
poses[0].as_rd_mol()

## single ligand section

In [ ]:
# Extract the 'sdf' column from the dataframe created earlier
sdf_column = df['sdf']

# Get the SDF data from the 3RD row
sdf_data = sdf_column.iloc[2]

# Write the SDF data to a file for pose analysis
with open('./data/Poses/SDF_poses/Ligand.sdf', 'w') as file:
    file.write(sdf_data)

In [ ]:
# convert the SDF file to a PDB file
supplier = Chem.SDMolSupplier("../Data/other/SDF_poses/Ligand.sdf", removeHs=False)
mol = supplier[0]  # Assuming you have one molecule in the file

# Write to PDB
with open("./data/Poses/PDB_poses/Ligand.pdb", "w") as f:
    f.write(Chem.MolToPDBBlock(mol))

In [ ]:
# Initialize PDBParser and PDBIO
parser = PDB.PDBParser(QUIET=True)
io = PDB.PDBIO()

# Load protein and ligand PDB files
protein_structure = parser.get_structure('protein', './Data/Receptor files/7vhy-noligand.pdb')
ligand_structure = parser.get_structure('ligand', './Data/Poses/PDB_poses/Ligand.pdb')

# Combine structures (using the first model)
protein_model = protein_structure[0]
ligand_model = ligand_structure[0]

# Ensure unique chain IDs for both structures
for chain in ligand_model:
    chain.id = 'L'  # Assign a unique chain ID for the ligand
    protein_model.add(chain)  # Add ligand chains to the protein model

# Save the combined structure
io.set_structure(protein_structure)  # Set the protein structure which now includes the ligand
io.save('./data/Poses docked with 7vhy/combined_structure.pdb')

## Multi pose section

In [ ]:
# loop over sdf column in filtered_df and write each sdf to a file and use inchikey as filename, if ducplicate, add a number to the filename
for index, row in filtered_df.iterrows():
    sdf_data = row['sdf']
    inchikey = row['inchikey'][:4]
    energy = row['vina_energy_total']
    with open(f'../Data/other/SDF poses/{inchikey}_{energy}.sdf', 'w') as file:
        file.write(sdf_data)

In [ ]:
# Load the first 10 ligands and obtain SDF files
# Extract the 'sdf' column from the dataframe
sdf_column = grouped_df['sdf']

# Loop over the first 10 rows and write each SDF to a separate file
for i in range(10):
    sdf_data = sdf_column.iloc[i]
    with open(f'./data/Poses/SDF_poses/Ligand_{i+1}.sdf', 'w') as file:
        file.write(sdf_data)

In [ ]:
# convert the obtained ligands to PDBs
for i in range(10):
    sdf_file = f'./data/Poses/SDF_poses/Ligand_{i+1}.sdf'
    pdb_file = f'./data/Poses/PDB_poses/Ligand_{i+1}.pdb'

    # Read the SDF file
    supplier = Chem.SDMolSupplier(sdf_file, removeHs=False)
    mol = supplier[0]  # Assuming one molecule per file

    # Write to PDB file
    with open(pdb_file, "w") as f:
        f.write(Chem.MolToPDBBlock(mol))

In [ ]:
# Unfortunately still manual untill further notice...
parser = PDB.PDBParser(QUIET=True)
io = PDB.PDBIO()

# Load protein and ligand PDB files
protein_structure = parser.get_structure('protein', './data/Receptor files/7vhy-noligand-nowater.pdb')
ligand_structure = parser.get_structure('ligand', './data/Poses/PDB_poses/Ligand_10.pdb')

# Combine structures (using the first model)
protein_model = protein_structure[0]
ligand_model = ligand_structure[0]

# Ensure unique chain IDs for both structures
for chain in ligand_model:
    chain.id = 'L'  # Assign a unique chain ID for the ligand
    protein_model.add(chain)  # Add ligand chains to the protein model

# Save the combined structure
io.set_structure(protein_structure)  # Set the protein structure which now includes the ligand
io.save('./data/Poses docked with 7vhy/combined_structure_10.pdb')

## Extra's

In [ ]:
view = nglview.show_rdkit(poses[0].as_rd_mol())  # load "3pqr" from RCSB PDB and display viewer widget
view

In [ ]:
poses[0].props

In [ ]:
complex = store.get_complex_for_pose(pose_id=poses[0].id)
complex

In [ ]:
complex_graph = store.targets[0].graph + poses[0].graph
complex_graph
plotly_molecular_graph(
    complex_graph.nx, 
    colour_nodes_by='entity', 
    colour_edges_by='type',
    plot_title='07dc2a5690da437bb34203e70e1c13a8_3 bound with 7vhy',
    figsize=(1000, 1000),
    node_size_min=10,
    node_size_multiplier=1,
    )

In [ ]:
# Check Vina energies of the top 10
grouped_df.head(10)['vina_energy_total']

In [ ]:
# get the 2D structure of the top 10 moleucles

# Extract the top 10 SMILES strings
top10_smiles = filtered_df['SMILES'].tolist()

# Convert to RDKit molecules
molecules = [Chem.MolFromSmiles(smiles) for smiles in top10_smiles]

# Format the pchembl_value_Mean column to 3 decimal places
filtered_df['vina_energy_total'] = filtered_df['vina_energy_total'].round(3)

# Update legends with formatted values
legends = filtered_df['vina_energy_total'].astype(str).tolist()

# Draw the molecules in a grid and save as PNG
# Draw the molecules in a grid with larger individual images
img = Draw.MolsToGridImage(molecules, molsPerRow=5, subImgSize=(400, 400), legends=legends)  # Increased size
img

In [ ]:
with open("../Results/images/Pharmacophore poses.png", "wb") as png:
    png.write(img.data)

In [3]:
from qsprpred.data import MoleculeTable
from qsprpred.data.descriptors.fingerprints import MorganFP


mt = MoleculeTable(
    store,
    name="TestMoleculeTable",
    path=store.baseDir,
)
mt.nJobs = os.cpu_count()
mt.chunkSize = 5

In [4]:
mt.getDF()

,ChemStorage_ID,parent_id,SMILES,sdf,original_smiles,SpockStorage_poses_representations_ID,SpockStorage_poses_representations_ID_before_change,target,vina_energy_inter,vina_energy_total,vina_energy_intra,vina_energy_torsions,vina_energy_intra_best_pose
SpockStorage_poses_representations_ID,,,,,,,,,,,,,
SpockStorage_poses_representations_library_f70260a4af4c458a8507e092278fe75c_0,None,ProtomerStorage_representations_library_9e8bfb...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,\n RDKit 3D\n\n 39 42 0 0 0 0...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,SpockStorage_poses_representations_library_f70...,SpockStorage_poses_representations_library_f70...,AF-Q13705-ACVR2B_-_prepared,-9.194,-8.022,-0.147,1.172,-0.147
SpockStorage_poses_representations_library_f70260a4af4c458a8507e092278fe75c_1,None,ProtomerStorage_representations_library_9e8bfb...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,\n RDKit 3D\n\n 39 42 0 0 0 0...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,SpockStorage_poses_representations_library_f70...,SpockStorage_poses_representations_library_f70...,AF-Q13705-ACVR2B_-_prepared,-8.592,-7.677,-0.354,1.122,-0.147
SpockStorage_poses_representations_library_f70260a4af4c458a8507e092278fe75c_2,None,ProtomerStorage_representations_library_9e8bfb...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,\n RDKit 3D\n\n 39 42 0 0 0 0...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,SpockStorage_poses_representations_library_f70...,SpockStorage_poses_representations_library_f70...,AF-Q13705-ACVR2B_-_prepared,-8.341,-7.641,-0.564,1.117,-0.147
SpockStorage_poses_representations_library_f70260a4af4c458a8507e092278fe75c_3,None,ProtomerStorage_representations_library_9e8bfb...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,\n RDKit 3D\n\n 39 42 0 0 0 0...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,SpockStorage_poses_representations_library_f70...,SpockStorage_poses_representations_library_f70...,AF-Q13705-ACVR2B_-_prepared,-8.148,-7.254,-0.314,1.06,-0.147
SpockStorage_poses_representations_library_f70260a4af4c458a8507e092278fe75c_4,None,ProtomerStorage_representations_library_9e8bfb...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,\n RDKit 3D\n\n 39 42 0 0 0 0...,[H]c1nc(N([H])[H])c2c(-c3c([H])c([H])c4oc(C([H...,SpockStorage_poses_representations_library_f70...,SpockStorage_poses_representations_library_f70...,AF-Q13705-ACVR2B_-_prepared,-8.91,-7.235,0.47,1.057,-0.147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SpockStorage_poses_representations_library_ba7d9e5ab8f9453da9a32f9bfa274286_0,None,ProtomerStorage_representations_library_d424a5...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,\n RDKit 3D\n\n 82 86 0 0 0 0...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,SpockStorage_poses_representations_library_ba7...,SpockStorage_poses_representations_library_ba7...,AF-Q13705-ACVR2B_-_prepared,-1.987,-1.302,3.219,0.685,3.219
SpockStorage_poses_representations_library_ba7d9e5ab8f9453da9a32f9bfa274286_1,None,ProtomerStorage_representations_library_d424a5...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,\n RDKit 3D\n\n 82 86 0 0 0 0...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,SpockStorage_poses_representations_library_ba7...,SpockStorage_poses_representations_library_ba7...,AF-Q13705-ACVR2B_-_prepared,1.737,1.058,3.097,-0.557,3.219
SpockStorage_poses_representations_library_ec5c21667ccf46adab6ba05c70e12ffd_0,None,ProtomerStorage_representations_library_d424a5...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,\n RDKit 3D\n\n 83 87 0 0 0 0...,[H]c1nc(N([H])c2c([H])c([H])c(N3C([H])([H])C([...,SpockStorage_poses_representations_library_ec5...,SpockStorage_poses_representations_library_ec5...,AF-Q13705-ACVR2B_-_prepared,3.639,2.385,-0.064,-1.255,-0.064


In [9]:
from descriptors import PLIPIFP

mt.addDescriptors([PLIPIFP(store.targets[0], n_poses=1)], recalculate=True)

qsprpred - WARNING - Could not convert descriptor values to '<class 'numpy.float32'>': could not convert string to float: 'SpockStorage_poses_representations_library_b84133a720cb4f3fbc73fb77bf0d8cce_0'
Keeping original types: hbonda_ASN_326_A        object
pication_LYS_325_A      object
hydroph_LYS_196_A       object
hydroph_ALA_197_A       object
hydroph_PHE_201_A       object
hydroph_LEU_328_A       object
hydroph_VAL_204_A       object
hydroph_ALA_215_A       object
hydroph_PHE_267_A       object
Pose_ID                 object
hbondd_LYS_325_A        object
hbondd_THR_265_A        object
hbondd_LYS_323_A        object
hbonda_ASP_321_A        object
hydroph_LYS_325_A       object
hbonda_ASP_275_A        object
pication_LYS_217_A      object
hbonda_LYS_325_A        object
hbondd_SER_272_A        object
hydroph_LEU_245_A       object
hydroph_THR_265_A       object
saltbridge_ASP_339_A    object
hbondd_ASN_326_A        object
hbonda_ARG_198_A        object
hydroph_THR_274_A       object

In [10]:
descs = mt.getDescriptors()
descs

,PLIPIFP_hbonda_ASN_326_A,PLIPIFP_pication_LYS_325_A,PLIPIFP_hydroph_LYS_196_A,PLIPIFP_hydroph_ALA_197_A,PLIPIFP_hydroph_PHE_201_A,PLIPIFP_hydroph_LEU_328_A,PLIPIFP_hydroph_VAL_204_A,PLIPIFP_hydroph_ALA_215_A,PLIPIFP_hydroph_PHE_267_A,PLIPIFP_Pose_ID,...,PLIPIFP_hbonda_THR_265_A,PLIPIFP_halogenbond_THR_265_A,PLIPIFP_hbondd_HIS_268_A,PLIPIFP_hbonda_HIS_268_A,PLIPIFP_hydroph_ALA_338_A,PLIPIFP_hbonda_ALA_266_A,PLIPIFP_hbondd_LYS_217_A,PLIPIFP_hbonda_LEU_263_A,PLIPIFP_hbonda_ALA_215_A,PLIPIFP_halogenbond_LYS_325_A
ChemStorage_ID,,,,,,,,,,,,,,,,,,,,,
VAARYSWULJUGST-UHFFFAOYSA-N,True,True,True,True,True,True,True,True,True,SpockStorage_poses_representations_library_b84...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYLKKYCXAOBSRM-UHFFFAOYSA-N,False,False,False,True,False,True,True,False,True,SpockStorage_poses_representations_library_495...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYLKKYCXAOBSRM-UHFFFAOYSA-N,False,True,False,True,True,True,True,False,False,SpockStorage_poses_representations_library_d92...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYLKKYCXAOBSRM-UHFFFAOYSA-N,False,True,False,True,True,True,True,False,False,SpockStorage_poses_representations_library_f85...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYLKKYCXAOBSRM-UHFFFAOYSA-N,False,False,False,True,False,True,True,False,True,SpockStorage_poses_representations_library_f2b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YYLKKYCXAOBSRM-UHFFFAOYSA-N,False,False,True,True,False,False,True,False,True,SpockStorage_poses_representations_library_efe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YZDJQTHVDDOVHR-UHFFFAOYSA-N,False,False,False,False,False,True,True,True,False,SpockStorage_poses_representations_library_dce...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
YZDJQTHVDDOVHR-UHFFFAOYSA-N,False,False,False,False,False,True,True,True,True,SpockStorage_poses_representations_library_3c6...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZBNZXTGUTAYRHI-UHFFFAOYSA-N,False,False,False,True,False,True,False,False,False,SpockStorage_poses_representations_library_277...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
mt.addDescriptors([MorganFP(radius=3, nBits=2048)], recalculate=True)

In [ ]:
mt.getDescriptors()

In [29]:
from descriptors_from_sdf import calc_plip_from_dir
df = calc_plip_from_dir("../Data/other/SDF poses", "../Data/target/AF-Q13705-ACVR2B_-_prepared.pdb")

In [30]:
df

,hbondd_HIS_268_A,hydroph_PHE_267_A,hydroph_LEU_328_A,hydroph_ASP_339_A,hydroph_LYS_196_A,hydroph_VAL_204_A,Pose_ID,hbonda_HIS_268_A,hydroph_ALA_197_A,hydroph_THR_265_A,...,saltbridge_GLU_230_A,saltbridge_LYS_325_A,hbondd_ARG_198_A,hbonda_SER_272_A,hbonda_ASP_321_A,hbonda_GLU_230_A,hbonda_ARG_198_A,hbondd_LYS_217_A,saltbridge_GLU_194_A,hydroph_GLU_194_A
0,True,True,True,True,True,True,B4B_5OXG,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,True,True,True,False,True,True,C9U_6JUX,True,True,True,...,False,False,False,False,False,False,False,False,False,False
0,True,True,False,True,False,False,H8H_6ZGC,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,True,False,True,True,True,False,XQX_8UWR,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,True,True,CUIH_-9.462,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,True,True,True,True,True,True,LDN_3Q4U,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,True,True,False,False,True,True,MSZ_6SZM,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,True,False,True,True,MFAQ_-8.681,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,True,True,False,False,True,True,TZX_8C7Z,False,True,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,True,False,True,True,QHKY_-8.946,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
# save df to csv
df.to_csv("../Data/docs/8.PLIP_poses.csv", index=False)